In [1]:
import node2vec
import pandas as pd

In [4]:
import os
import numpy as np
import networkx as nx
from collections import Counter

In [2]:
train_dataset = pd.read_csv("./conceptnet_modify_update.csv", index_col=0, encoding='utf-8')

c:\users\qkrwo\anaconda3\envs\node2vec\lib\site-packages\numpy\lib\arraysetops.py:583: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [3]:
train_dataset.head(10)

,relation,subject,value
0,Antonym,0,1
1,Antonym,12_hour_clock,24_hour_clock
2,Antonym,24_hour_clock,12_hour_clock
3,Antonym,5,3
4,Antonym,a.c,d.c
5,Antonym,a.m,afternoon
6,Antonym,a.m,pm
7,Antonym,ab_extra,ab_intra
8,Antonym,ab_intra,ab_extra
9,Antonym,abactinal,actinal


In [5]:
def add_nodes(G, df, col, type_name):
    """Add entities to G from the 'col' column of the 'df' DataFrame. The new nodes are annotated with 'type_name' label."""
    nodes = list(df[~df[col].isnull()][col].unique())
    G.add_nodes_from([(n,dict(type=type_name)) for n in nodes])
    print("Nodes (%s,%s) were added" % (col, type_name))
    
def add_nodes_2col(G, df, col1, col2, type_name):
    """Add entities to G from the 'col' column of the 'df' DataFrame. The new nodes are annotated with 'type_name' label."""
    nodes = list((df[~df[col1].isnull()][col1] + df[~df[col2].isnull()][col2]).unique())
    print(len(nodes))
    G.add_nodes_from([(n,dict(type=type_name)) for n in nodes])
    print("Nodes (%s,%s) were added" % (col1, type_name))
    print("Nodes (%s,%s) were added" % (col2, type_name))
    print(word_nodes)
    
def add_links(G, df, col1, col2, type_name):
    """Add links to G from the 'df' DataFrame. The new edges are annotated with 'type_name' label."""
    df_tmp = df[(~df[col1].isnull()) & (~df[col2].isnull())]
    links = list(zip(df_tmp[col1],df_tmp[col2]))
    G.add_edges_from([(src, trg, dict(type=type_name)) for src, trg in links])
    print("Edges (%s->%s,%s) were added" % (col1, col2, type_name))
    
def add_relation(G, df, col1, col2, relation_name):
    """Add links to G from the 'df' DataFrame. The new edges are annotated with 'type_name' label."""
    df_tmp = df[~(df['relation'] != relation_name)]
    links = list(zip(df_tmp[col1],df_tmp[col2]))
    G.add_edges_from([(src, trg, dict(type=relation_name)) for src, trg in links])
    print("Edges (%s->%s,%s) were added" % (col1, col2, relation_name))

    
def add_relation_words(G, df, col1, col2, relation_name):
    """Add links to G from the 'df' DataFrame. The new edges are annotated with 'type_name' label."""
    df_tmp = df[~(df['relation'] != relation_name)]
    links = list(zip(df_tmp[col1],df_tmp[col2]))
    word_links = []
    rel_links = []
    for src, trg in links:
        src, trg = str(src), str(trg)
        src_words = src.split("_")
        trg_words = trg.split("_")
        for i in range(len(src_words) - 1) :
            word_links.append((src_words[i], src_words[i+1], dict(type="sub_continous")))
        rel_links.append((src_words[-1], trg_words[0], dict(type=relation_name)))
        for i in range(len(trg_words) - 1) :
            word_links.append((trg_words[i], trg_words[i+1], dict(type="val_continous")))
    G.add_edges_from(word_links)
    G.add_edges_from(rel_links)
    print("relation number = ", len(rel_links))
    print("Edges (%s->%s,%s) were added" % (col1, col2, relation_name))
    
def add_relation_words2(G, df, col1, col2, relation_name):
    """Add links to G from the 'df' DataFrame. The new edges are annotated with 'type_name' label."""
    df_tmp = df[~(df['relation'] != relation_name)]
    links = list(zip(df_tmp[col1],df_tmp[col2]))
    word_links = []
    rel_links = []
    for src, trg in links:
        src, trg = str(src), str(trg)
        rel_links.append((src.split("/")[0], trg.split("/")[0], dict(type=relation_name)))
    G.add_edges_from(rel_links)
    print("Edges (%s->%s,%s) were added %s" % (col1, col2, relation_name, len(rel_links)))


In [7]:
list(train_dataset[~train_dataset['relation'].isnull()]['relation'].unique())

['Antonym',
 'AtLocation',
 'DerivedFrom',
 'DistinctFrom',
 'Entails',
 'EtymologicallyDerivedFrom',
 'EtymologicallyRelatedTo',
 'FormOf',
 'HasContext',
 'InstanceOf',
 'IsA',
 'MannerOf',
 'PartOf',
 'RelatedTo',
 'SimilarTo',
 'SymbolOf',
 'Synonym',
 'UsedFor',
 'dbpedia/capital',
 'dbpedia/field',
 'dbpedia/genre',
 'dbpedia/genus',
 'dbpedia/influencedBy',
 'dbpedia/knownFor',
 'dbpedia/language',
 'dbpedia/leader',
 'dbpedia/occupation',
 'dbpedia/product']

In [42]:
subjects = np.array((train_dataset[~train_dataset['subject'].isnull()]['subject']).unique())

In [41]:
values = np.array(train_dataset[~train_dataset['value'].isnull()]['value'].unique())

In [48]:
vocabs = np.concatenate((subjects,values),axis=0)

In [90]:
array = np.array([[]])
array.shape
array[0] = [1,2,3,4]

ValueError: could not broadcast input array from shape (4,) into shape (0,)

In [96]:
def tokenize_words(words):
    return_tokens = np.array([])
    i = 0 
    j = 0
    tokens= np.array([])
    for token in words:
        token = str(token)
        phrase = token.split('_')
        tokens = np.append(tokens, phrase, axis = 0)
        i = i + 1
        if i % 1000 == 0:
            print("not working at (i, size)", i, words.shape[0])
        if i % 5000 == 0:
            j = j + 1
            return_tokens = np.concatenate((return_tokens, tokens), axis = 0)
            tokens = np.array([])
    return_tokens = np.concatenate((return_tokens, tokens), axis = 0)
    return return_tokens

In [97]:
value_vocabs = tokenize_words(values)
subject_vocabs = tokenize_words(subjects)

not working at (i, size) 1000 516597
not working at (i, size) 2000 516597
not working at (i, size) 3000 516597
not working at (i, size) 4000 516597
not working at (i, size) 5000 516597
not working at (i, size) 6000 516597
not working at (i, size) 7000 516597
not working at (i, size) 8000 516597
not working at (i, size) 9000 516597
not working at (i, size) 10000 516597
not working at (i, size) 11000 516597
not working at (i, size) 12000 516597
not working at (i, size) 13000 516597
not working at (i, size) 14000 516597
not working at (i, size) 15000 516597
not working at (i, size) 16000 516597
not working at (i, size) 17000 516597
not working at (i, size) 18000 516597
not working at (i, size) 19000 516597
not working at (i, size) 20000 516597
not working at (i, size) 21000 516597
not working at (i, size) 22000 516597
not working at (i, size) 23000 516597
not working at (i, size) 24000 516597
not working at (i, size) 25000 516597
not working at (i, size) 26000 516597
not working at (i, si

not working at (i, size) 219000 516597
not working at (i, size) 220000 516597
not working at (i, size) 221000 516597
not working at (i, size) 222000 516597
not working at (i, size) 223000 516597
not working at (i, size) 224000 516597
not working at (i, size) 225000 516597
not working at (i, size) 226000 516597
not working at (i, size) 227000 516597
not working at (i, size) 228000 516597
not working at (i, size) 229000 516597
not working at (i, size) 230000 516597
not working at (i, size) 231000 516597
not working at (i, size) 232000 516597
not working at (i, size) 233000 516597
not working at (i, size) 234000 516597
not working at (i, size) 235000 516597
not working at (i, size) 236000 516597
not working at (i, size) 237000 516597
not working at (i, size) 238000 516597
not working at (i, size) 239000 516597
not working at (i, size) 240000 516597
not working at (i, size) 241000 516597
not working at (i, size) 242000 516597
not working at (i, size) 243000 516597
not working at (i, size) 

not working at (i, size) 430000 516597
not working at (i, size) 431000 516597
not working at (i, size) 432000 516597
not working at (i, size) 433000 516597
not working at (i, size) 434000 516597
not working at (i, size) 435000 516597
not working at (i, size) 436000 516597
not working at (i, size) 437000 516597
not working at (i, size) 438000 516597
not working at (i, size) 439000 516597
not working at (i, size) 440000 516597
not working at (i, size) 441000 516597
not working at (i, size) 442000 516597
not working at (i, size) 443000 516597
not working at (i, size) 444000 516597
not working at (i, size) 445000 516597
not working at (i, size) 446000 516597
not working at (i, size) 447000 516597
not working at (i, size) 448000 516597
not working at (i, size) 449000 516597
not working at (i, size) 450000 516597
not working at (i, size) 451000 516597
not working at (i, size) 452000 516597
not working at (i, size) 453000 516597
not working at (i, size) 454000 516597
not working at (i, size) 

not working at (i, size) 129000 912210
not working at (i, size) 130000 912210
not working at (i, size) 131000 912210
not working at (i, size) 132000 912210
not working at (i, size) 133000 912210
not working at (i, size) 134000 912210
not working at (i, size) 135000 912210
not working at (i, size) 136000 912210
not working at (i, size) 137000 912210
not working at (i, size) 138000 912210
not working at (i, size) 139000 912210
not working at (i, size) 140000 912210
not working at (i, size) 141000 912210
not working at (i, size) 142000 912210
not working at (i, size) 143000 912210
not working at (i, size) 144000 912210
not working at (i, size) 145000 912210
not working at (i, size) 146000 912210
not working at (i, size) 147000 912210
not working at (i, size) 148000 912210
not working at (i, size) 149000 912210
not working at (i, size) 150000 912210
not working at (i, size) 151000 912210
not working at (i, size) 152000 912210
not working at (i, size) 153000 912210
not working at (i, size) 

not working at (i, size) 340000 912210
not working at (i, size) 341000 912210
not working at (i, size) 342000 912210
not working at (i, size) 343000 912210
not working at (i, size) 344000 912210
not working at (i, size) 345000 912210
not working at (i, size) 346000 912210
not working at (i, size) 347000 912210
not working at (i, size) 348000 912210
not working at (i, size) 349000 912210
not working at (i, size) 350000 912210
not working at (i, size) 351000 912210
not working at (i, size) 352000 912210
not working at (i, size) 353000 912210
not working at (i, size) 354000 912210
not working at (i, size) 355000 912210
not working at (i, size) 356000 912210
not working at (i, size) 357000 912210
not working at (i, size) 358000 912210
not working at (i, size) 359000 912210
not working at (i, size) 360000 912210
not working at (i, size) 361000 912210
not working at (i, size) 362000 912210
not working at (i, size) 363000 912210
not working at (i, size) 364000 912210
not working at (i, size) 

not working at (i, size) 556000 912210
not working at (i, size) 557000 912210
not working at (i, size) 558000 912210
not working at (i, size) 559000 912210
not working at (i, size) 560000 912210
not working at (i, size) 561000 912210
not working at (i, size) 562000 912210
not working at (i, size) 563000 912210
not working at (i, size) 564000 912210
not working at (i, size) 565000 912210
not working at (i, size) 566000 912210
not working at (i, size) 567000 912210
not working at (i, size) 568000 912210
not working at (i, size) 569000 912210
not working at (i, size) 570000 912210
not working at (i, size) 571000 912210
not working at (i, size) 572000 912210
not working at (i, size) 573000 912210
not working at (i, size) 574000 912210
not working at (i, size) 575000 912210
not working at (i, size) 576000 912210
not working at (i, size) 577000 912210
not working at (i, size) 578000 912210
not working at (i, size) 579000 912210
not working at (i, size) 580000 912210
not working at (i, size) 

not working at (i, size) 768000 912210
not working at (i, size) 769000 912210
not working at (i, size) 770000 912210
not working at (i, size) 771000 912210
not working at (i, size) 772000 912210
not working at (i, size) 773000 912210
not working at (i, size) 774000 912210
not working at (i, size) 775000 912210
not working at (i, size) 776000 912210
not working at (i, size) 777000 912210
not working at (i, size) 778000 912210
not working at (i, size) 779000 912210
not working at (i, size) 780000 912210
not working at (i, size) 781000 912210
not working at (i, size) 782000 912210
not working at (i, size) 783000 912210
not working at (i, size) 784000 912210
not working at (i, size) 785000 912210
not working at (i, size) 786000 912210
not working at (i, size) 787000 912210
not working at (i, size) 788000 912210
not working at (i, size) 789000 912210
not working at (i, size) 790000 912210
not working at (i, size) 791000 912210
not working at (i, size) 792000 912210
not working at (i, size) 

In [99]:
value_vocabs

array(['1', '24', 'hour', ..., 'boot', 'management', 'consulting'],
      dtype='<U85')

In [100]:
vocabs_fin = np.concatenate((value_vocabs, subject_vocabs), axis = 0)

In [104]:
vocabs_set = np.unique(vocabs_fin)

In [115]:
print("before\t  \t after\t  \tdecreased")
print(vocabs_fin.shape, "\t", vocabs_set.shape, "\t", vocabs_fin.shape[0] - vocabs_set.shape[0])

before	  	 after	  	decreased
(2057885,) 	 (706317,) 	 1351568


In [146]:
def apostrophe_tokenizer(words):
    return_tokens = np.array([])
    i = 0 
    j = 0
    tokens= np.array([])
    for token in words:
        token = str(token)
        phrase = token.split("\'")
        phrase2 = token.split("’")
        if(True):
            tokens = np.append(tokens, phrase, axis = 0)
        else:
            print(phrase)
            for index in range(1, len(phrase)):
                phrase[index] = "\'" + phrase[index]
            print(phrase)
            tokens = np.append(tokens, phrase, axis = 0)
        i = i + 1
        if i % 1000 == 0:
            print("not working at (i, size)", i, words.shape[0])
        if i % 5000 == 0:
            j = j + 1
            return_tokens = np.concatenate((return_tokens, tokens), axis = 0)
            tokens = np.array([])
    return_tokens = np.concatenate((return_tokens, tokens), axis = 0)
    return return_tokens


In [147]:
vocab_apo2 = apostrophe_tokenizer(vocabs_set)

not working at (i, size) 1000 706317
not working at (i, size) 2000 706317
not working at (i, size) 3000 706317
not working at (i, size) 4000 706317
not working at (i, size) 5000 706317
not working at (i, size) 6000 706317
not working at (i, size) 7000 706317
not working at (i, size) 8000 706317
not working at (i, size) 9000 706317
not working at (i, size) 10000 706317
not working at (i, size) 11000 706317
not working at (i, size) 12000 706317
not working at (i, size) 13000 706317
not working at (i, size) 14000 706317
not working at (i, size) 15000 706317
not working at (i, size) 16000 706317
not working at (i, size) 17000 706317
not working at (i, size) 18000 706317
not working at (i, size) 19000 706317
not working at (i, size) 20000 706317
not working at (i, size) 21000 706317
not working at (i, size) 22000 706317
not working at (i, size) 23000 706317
not working at (i, size) 24000 706317
not working at (i, size) 25000 706317
not working at (i, size) 26000 706317
not working at (i, si

not working at (i, size) 216000 706317
not working at (i, size) 217000 706317
not working at (i, size) 218000 706317
not working at (i, size) 219000 706317
not working at (i, size) 220000 706317
not working at (i, size) 221000 706317
not working at (i, size) 222000 706317
not working at (i, size) 223000 706317
not working at (i, size) 224000 706317
not working at (i, size) 225000 706317
not working at (i, size) 226000 706317
not working at (i, size) 227000 706317
not working at (i, size) 228000 706317
not working at (i, size) 229000 706317
not working at (i, size) 230000 706317
not working at (i, size) 231000 706317
not working at (i, size) 232000 706317
not working at (i, size) 233000 706317
not working at (i, size) 234000 706317
not working at (i, size) 235000 706317
not working at (i, size) 236000 706317
not working at (i, size) 237000 706317
not working at (i, size) 238000 706317
not working at (i, size) 239000 706317
not working at (i, size) 240000 706317
not working at (i, size) 

not working at (i, size) 430000 706317
not working at (i, size) 431000 706317
not working at (i, size) 432000 706317
not working at (i, size) 433000 706317
not working at (i, size) 434000 706317
not working at (i, size) 435000 706317
not working at (i, size) 436000 706317
not working at (i, size) 437000 706317
not working at (i, size) 438000 706317
not working at (i, size) 439000 706317
not working at (i, size) 440000 706317
not working at (i, size) 441000 706317
not working at (i, size) 442000 706317
not working at (i, size) 443000 706317
not working at (i, size) 444000 706317
not working at (i, size) 445000 706317
not working at (i, size) 446000 706317
not working at (i, size) 447000 706317
not working at (i, size) 448000 706317
not working at (i, size) 449000 706317
not working at (i, size) 450000 706317
not working at (i, size) 451000 706317
not working at (i, size) 452000 706317
not working at (i, size) 453000 706317
not working at (i, size) 454000 706317
not working at (i, size) 

not working at (i, size) 646000 706317
not working at (i, size) 647000 706317
not working at (i, size) 648000 706317
not working at (i, size) 649000 706317
not working at (i, size) 650000 706317
not working at (i, size) 651000 706317
not working at (i, size) 652000 706317
not working at (i, size) 653000 706317
not working at (i, size) 654000 706317
not working at (i, size) 655000 706317
not working at (i, size) 656000 706317
not working at (i, size) 657000 706317
not working at (i, size) 658000 706317
not working at (i, size) 659000 706317
not working at (i, size) 660000 706317
not working at (i, size) 661000 706317
not working at (i, size) 662000 706317
not working at (i, size) 663000 706317
not working at (i, size) 664000 706317
not working at (i, size) 665000 706317
not working at (i, size) 666000 706317
not working at (i, size) 667000 706317
not working at (i, size) 668000 706317
not working at (i, size) 669000 706317
not working at (i, size) 670000 706317
not working at (i, size) 

In [140]:
print("before\t  \t after\t  \tincreased")
print(vocabs_set.shape, "\t", vocab_apo.shape, "\t", vocab_apo.shape[0] - vocabs_set.shape[0])

before	  	 after	  	increased
(706317,) 	 (709321,) 	 3004


In [148]:
vocabs_appo_fin2 = np.unique(vocab_apo2)

In [143]:
print("before\t  \t after\t  \tdecreased")
print(vocabs_set.shape, "\t", vocabs_appo_fin.shape, "\t", vocabs_set.shape[0] - vocabs_appo_fin.shape[0])

before	  	 after	  	decreased
(706317,) 	 (704131,) 	 2186


In [145]:
np.savetxt("tokenized_vocabs", vocabs_appo_fin, fmt="%s", encoding="utf-8")

In [150]:
np.savetxt("tokenized_vocabs_appo.txt", vocabs_appo_fin2, fmt="%s", encoding="utf-8")

In [152]:
vocabs_appo_fin2

array(['0', '00', '000', ..., '👜', '🕈', '🛅'], dtype='<U92')